In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.2.0'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Wireless_v1_00.tsv.gz"
spark.sparkContext.addFile(url)

In [ ]:
from pyspark.sql.functions import to_date
import pyspark.sql.functions as func

In [ ]:
# Read in the Review dataset as a DataFrame
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get(""), sep="\t", header=True, inferSchema=True)
df.show()

In [ ]:
# Create the vine_table. DataFrame
vine_df = df.select(['review_id', 'star_rating', 'helpful_votes', 'total_votes', 'vine', 'verified_purchase'])
vine_df.show()

In [ ]:
# Filter DataFrame for rows with 20 or more votes
vine_df1 = vine_df.filter(vine_df.total_votes >= 20)
vine_df1.show()

In [ ]:
# Filter last DF where the number of helpful_votes divided by total_votes is equal to or greater than 50%.
helpful_df = vine_df1.filter((vine_df1.helpful_votes / vine_df1.total_votes) >= 0.5)

In [ ]:
# Filter last DF for paid reviews
paid_df = helpful_df.filter(helpful_df.vine == 'Y')
paid_df.show()

In [ ]:
# Filter for unpaid reviews
unpaid_df = helpful_df.filter(helpful_df.vine == 'N')
unpaid_df.show()

In [ ]:
# Count total reviews
vine_count = vine_df.count()
vine_count

In [ ]:
# Count helpful reviews
helpful_count = helpful_df.count()
helpful_count

In [ ]:
# Count 5* reviews
helpful_5_count = helpful_df.filter(helpful_df.star_rating == 5).count()
helpful_5_count

In [ ]:
# Paid reviews 
paid_count = paid_df.count()
paid_count

In [ ]:
# Count paid 5* reviews
paid_5_star = paid_df.filter(paid_df.star_rating == 5).count()
paid_5_star

In [ ]:
# Percentage of 5* paid reviews
percentage_5_paid = paid_5_star / paid_count * 100
percentage_5_paid

In [ ]:
# Unpaid reviews
unpaid_count = unpaid_df.count()
unpaid_count

In [ ]:
# Count unpaid 5* reviews
unpaid_5_star = unpaid_df.filter(unpaid_df.star_rating == 5).count()
unpaid_5_star

In [ ]:
# Percentage of 5* unpaid reviews
percentage_5_unpaid = unpaid_5_star / unpaid_count * 100
percentage_5_unpaid